# 4: EET and Cross Compartment Electron Shuttles
## This notebook shows the code and calculations used to determine:
### - EET flux versus QF
### - Energetic coupling between the plastid and mitochondria

In [ ]:
import pandas as pd
import numpy as np
import csv
import cobra
from cobra.core.metabolite import elements_and_molecular_weights
elements_and_molecular_weights['R']=0.0
elements_and_molecular_weights['Z']=0.0
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
import fcns.modeling_functions as fcns

In [ ]:
## This cell will recreate Figure 4 from the manuscript.
## This sets the QF to an increasing value and calculates the EET in the system at each QF value
## Both the fully constrained [Y(II), D1 repair, photon constraints, Oxygen exchange] and the 
## previous modeling with up to 20% of photons allowed to leave upstream of the photosystems are 
## included, as in Figure 4.

base_model = cobra.io.load_json_model('HL_base_iLB1035.json')
(HL_YII_eetLET_out, HL_YII_QF, HL_YII_LET) = fcns.sim_eet(base_model,'HL',DM_flag = False, ngam_comp = 'mito')
(HL_DM20_eetLET_out, HL_DM20_QF, HL_DM20_LET) = fcns.sim_eet(base_model,'HL',DM_flag = True, ngam_comp = 'mito')

base_model = cobra.io.load_json_model('LL_base_iLB1035.json')
(LL_YII_eetLET_out, LL_YII_QF, LL_YII_LET) = fcns.sim_eet(base_model,'LL',DM_flag = False, ngam_comp = 'mito')
(LL_DM20_eetLET_out, LL_DM20_QF, LL_DM20_LET) = fcns.sim_eet(base_model,'LL',DM_flag = True, ngam_comp = 'mito')



In [ ]:
## Figure 4

QF_range = np.linspace(1e-10,1.2e-9,21)
plot = sns.scatterplot(x=QF_range, y=LL_DM20_eetLET_out, marker = '^', edgecolor="tab:orange", facecolors="none", linewidth = 2, s=70,label = 'LL Photon 20%')
plot = sns.scatterplot(x=QF_range, y=HL_DM20_eetLET_out, marker = 'o', edgecolor="blue", facecolors="none", s=70, linewidth = 2, label = 'HL Photon 20%')
plot = sns.scatterplot(x=QF_range, y=LL_YII_eetLET_out, marker = '^', edgecolor="tab:orange", facecolors="tab:orange", linewidth = 1, s=70, label = 'LL Y(II)+D1repair')
plot = sns.scatterplot(x=QF_range, y=HL_YII_eetLET_out, marker = 'o', edgecolor="blue", facecolors="blue", s=70, label = 'HL Y(II)+D1repair')
plot.set(xlim = (0,1.22e-9),xlabel='Quantum Flux (umol photons cell-1 s-1)',ylabel = 'EET Flux (mmol e- gDW-1 h-1)')

## Energetic coupling between the plastid and mitochondrion
### Figure 4B
### High light flux comparison between Y(II), 20% Photon loss, and NGAM routed
### to the plastid instead of the mitochondrion (PTOX instead of AOXm)

In [ ]:
base_model = cobra.io.load_json_model('HL_base_iLB1035.json')
(yII_biomass, yII_model) = fcns.simulate(base_model,'HL',photon_const=True,Po_const=True,
                                         YII_const=True,D1_const=True,DM20=False,ngam_comp='mito')
(DM20_biomass, DM20_model) = fcns.simulate(base_model,'HL',photon_const=False,Po_const=True,
                                         YII_const=False,D1_const=False,DM20=True,ngam_comp='mito')
(PTOX_biomass, PTOX_model) = fcns.simulate(base_model,'HL',photon_const=True,Po_const=True,
                                         YII_const=True,D1_const=True,DM20=False,ngam_comp='chloro')

In [ ]:
## Model reactions that are associated with specific pathways
xcomp_rxns = {'TCA' : ['ICDH_m','IDH_m','OGDH_E2_m','SDH_m','ME1_m','ME2_m','MDH_m','GLUDH2_m'],
              'Glyco_m' : ['GAPDH_m','PDH_E2_m'],
              'Lysine' : ['OADPDH_m','GCDH_m','HADH40_m'],
              'PRred' : ['GCS_m'],
              'BCAA' : ['BCKDHI_m','BCKDHV_m','BCKDHL_m','IBCOAD_m','ISOVCOADH_m','2MBCOAD_m','3H2MBCOAOR_m','3HIBDH_m','MMSAD_m','MMM_m'],
              'Orn' : ['PRODH_m','P5CDH_m'],
              'AOX' : ['AOX_m','NGAM'],
              'ROS' : ['GDR_m','GDR_NADP_m','CCP2_m'],
              'PRox' : ['GLYTA_m','AGAT_m']}


TI = 20. #time interval in minutes

In [ ]:
yII_fluxes = fcns.calc_fluxes(yII_model,xcomp_rxns,gDW=yII_biomass['700']['Biomass'],TI=TI)
DM20_fluxes = fcns.calc_fluxes(DM20_model,xcomp_rxns,gDW=DM20_biomass['700']['Biomass'],TI=TI)
PTOX_fluxes = fcns.calc_fluxes(PTOX_model,xcomp_rxns,gDW=PTOX_biomass['700']['Biomass'],TI=TI)

In [ ]:
yII_df = pd.DataFrame()
rxns = [k for k,v in yII_fluxes.items()]
fluxes = [v for k,v in yII_fluxes.items()]
condition = 'Y(II)+D1'

yII_df['rxns']=rxns
yII_df['fluxes']=fluxes
yII_df['condition']=[condition]*len(rxns)

DM20_df = pd.DataFrame()
rxns = [k for k,v in DM20_fluxes.items()]
fluxes = [v for k,v in DM20_fluxes.items()]
condition = 'Photon20%'

DM20_df['rxns']=rxns
DM20_df['fluxes']=fluxes
DM20_df['condition']=[condition]*len(rxns)

PTOX_df = pd.DataFrame()
rxns = [k for k,v in PTOX_fluxes.items()]
fluxes = [v for k,v in PTOX_fluxes.items()]
condition = 'PTOX'

PTOX_df['rxns']=rxns
PTOX_df['fluxes']=fluxes
PTOX_df['condition']=[condition]*len(rxns)

data_df = pd.concat([DM20_df,yII_df,PTOX_df])

In [ ]:
sns.barplot(x ='rxns',y='fluxes', hue = 'condition',data = data_df, palette='YlGnBu_r')